这些天在看David Beazley的[Python3 metaprogramming](https://www.youtube.com/watch?v=sPiWg5jSoZI)视频，觉得是时候总结下视频中学到的内容

## 装饰器

首先看一个需求，就是想在函数被调用时，记录一下，可以简单的加个print

In [33]:
def add(x, y):
    print('add')
    return x + y

def sub(x, y):
    print('sub')
    return x - y

def mul(x, y):
    print('mul')
    return x * y

def div(x, y):
    print('div')
    return x / y

print(add(3, 5))
print(sub(5, 2))

add
8
sub
3


但是这样的话, print语句就重复了, 每个函数里都得加print, 于是我们可以使用装饰器

In [34]:
# 一个简单的装饰器
def debug(func):
    def wrapper(*args, **kwargs):
        print(func.__name__)
        return func(*args, **kwargs)
    return wrapper

In [17]:
print(debug)

<function debug at 0x10370f840>


In [35]:
@debug
def add(x, y):
    return x + y

@debug
def sub(x, y):
    return x - y

@debug
def mul(x, y):
    return x * y

@debug
def div(x, y):
    return x / y

print(add(3, 5))
print(sub(5, 2))

add
8
sub
3


### 使用functools

但是这个简单的装饰器是存在问题的，它会忽略被装饰的函数

In [19]:
print(add)

<function debug.<locals>.wrapper at 0x1037a6bf8>


这里add函数经过debug装饰器装饰后，函数名都被忽略了，这个时候functools模块就派上用场了, 里面的wraps装饰器就是用来解决这个问题

In [36]:

from functools import wraps

def debug(func):
    msg = func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)
    return wrapper

In [21]:
@debug
def add(x, y):
    return x + y

@debug
def sub(x, y):
    return x - y

@debug
def mul(x, y):
    return x * y

@debug
def div(x, y):
    return x / y

print(add(3, 5))
print(sub(5, 2))

add
8
sub
3


In [22]:
print(add)

<function add at 0x1037afa60>


### 带参数的装饰器

In [39]:
# decorators with args
from functools import wraps

def debug(prefix=''):
    def decroate(func):
        msg = prefix + func.__qualname__
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(msg)
            return func(*args, **kwargs)
        return wrapper
    return decroate

@debug('###')
def add(x, y):
    return x + y

add(3, 2)

###add


5

这种带参数的装饰器有一个头疼的问题是, 使用装饰器时, 如果不想传参数, 也得加上括号, 不然会报错

In [40]:
@debug
def sub(x, y):
    return x - y

sub(5, 3)

TypeError: decroate() takes 1 positional argument but 2 were given

加上括号后, 就不报错, 这很丑陋

In [41]:
@debug()
def sub(x, y):
    return x - y

sub(5, 3)

sub


2

这里有一个小技巧, 实现如下

In [42]:
from functools import wraps, partial

def debug(func=None, prefix=''):
    if func is None:
        return partial(debug, prefix=prefix)
    msg = prefix + func.__qualname__
    @wraps(func)
    def wrapper(*args, **kwargs):     
        print(msg)
        return func(*args, **kwargs)
    return wrapper


如此, 当使用默认参数时, 即便不带括号时，也不会报错

In [43]:
@debug(prefix='###')
def add(x, y):
    return x + y


@debug
def sub(x, y):
    return x - y

print(add(3, 2))
print(sub(5, 3))

###add
5
sub
2


### 类装饰器

以下我们定义一个Spam类,

In [ ]:
class Spam:
    def a(self):
        pass
    def b(self):
        pass
    @classmethod
    def c(cls):
        pass
    @staticmethod
    def d():
        pass

然后我们想在类的方法被调用时, 能够记录下, 想上面的函数被调用时一样, 这时我们就会可以编写一个类装饰器

In [46]:
def debugmethods(cls):
    # cls is class
    for key, val in vars(cls).items():
        if callable(val):
            setattr(cls, key, debug(val))
    return cls

@debugmethods
class Spam:
    def a(self):
        pass
    def b(self):
        pass
    @classmethod
    def c(cls):
        pass
    @staticmethod
    def d():
        pass

spam = Spam()
spam.a()
spam.b()
spam.c()
spam.d()

Spam.a
Spam.b


这里只打印了a和b, 没有打印c和d, 这什么原因呢? 这是因为classmethod和staticmethod都是descriptor, 也就是描述器, 它们没有实现`__call__`方法，也就不是callable的

In [ ]:
vars(Spam)

我们也可以编写一个类装饰器, 当获取一个属性时, 打印日志

In [47]:
# debug access
def debugattr(cls):
    orig_getattribute = cls.__getattribute__
    def __getattribute__(self, name):
        print('Get:', name)
        return orig_getattribute(self, name)
    cls.__getattribute__ = __getattribute__
    return cls

@debugattr
class Spam:
    def __init__(self, x, y):
        self.x = x
        self.y = y

spam = Spam(2, 3)
spam.x

Get: x


2

## metaclass

现在我们想对所有的类都能打印日志，一个解决的办法是在所有的类前面都加上类装饰器

In [48]:
@debugmethods
class Base:
    def a(self):
        pass
    def b(self):
        pass
    
@debugmethods
class Spam(Base):
    def a(self):
        pass
b = Base()
b.a()
s = Spam()
s.a()

Base.a
Spam.a


但这样很麻烦，于是metaclass派上用场了, metaclass最强的地方是可以控制类的创建

In [49]:
# a metaclass
class debugmeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsobj = super().__new__(cls, clsname, bases, clsdict)
#         print("here", cls, clsname, type(clsobj), clsobj)
        clsobj = debugmethods(clsobj)
#         print(vars(clsobj))
        return clsobj

class Base(metaclass=debugmeta):
    def a(self):
        pass
    def b(self):
        pass

    
class Spam(Base):
    def __init__(self, name):
        self.name = name
        
    def a(self):
        pass
    
b = Base()
b.a()
s = Spam('name')
s.a()

Base.a
Spam.__init__
Spam.a


从上面的例子中我们看到，有一个类有metaclass, 它的所有子类都有metaclass, 这说明metaclass是会被继承的。结合蔡元楠的《metaclass, 是潘多拉魔盒还是阿拉丁神灯》, 可以知道，这里debugmeta其实不只一种写法，在`__init__`函数里实现也是可以的。重载`__init__`的实现如下

In [50]:
# a metaclass
class debugmeta(type):
     def __init__(cls, name, bases, kwds):
        super(debugmeta, cls).__init__(name, bases, kwds)
        cls = debugmethods(cls)

class Base(metaclass=debugmeta):
    def a(self):
        pass
    def b(self):
        pass

    
class Spam(Base):
    def __init__(self, name):
        self.name = name
        
    def a(self):
        pass
b = Base()
b.a()
s = Spam('lala')
s.a()

Base.a
Spam.__init__
Spam.a


这是因为, 所有的类都是type的实例, 都是对type的`__call__`方法进行重载, 而type的`__call__`方法会调用`type.__new__(typeclass, classname, superclasses, attributedict)`和`type.__init__(class, classname, superclasses, attributedict)`, 所以上面重写`__new__`和重写`__init__`都是可以的。

写到这里，我脑海里冒出了一个想法，就是为啥这里一定要用metaclass呢? 用继承的方式难道不行吗？于是自己尝试写了个继承的方式, 发现也是跑得通的。

In [51]:
# a baseclass
class debugmeta:
    def __new__(cls, *args, **kwargs):
        cls = debugmethods(cls)
        clsobj = object.__new__(cls)
#         clsobj = debugmethods(clsobj)
#         print('aaa', type(clsobj), clsobj)
#         print('aaa', clsobj, type(clsobj), vars(clsobj), dir(clsobj), type(clsobj.a), callable(clsobj.a))
        return clsobj
    
class Base(debugmeta):
    def a(self):
        pass

    
class Spam(Base):
    def __init__(self, name):
        self.name = name
    def a(self):
        pass

    
b = Base()
b.a()
s = Spam('name')
s.a()


Base.a
Spam.__init__
Spam.a


既然如此，那么蔡元楠在《metaclass, 是潘多拉魔盒还是阿拉丁神灯》介绍的yaml的动态序列化和逆序列化的能力又为何要用metaclass实现呢？用继承难道不行吗？于是也写了一个继承的版本, 代码如下。

In [52]:
import yaml


class MyYAMLObjectBaseclass(object):
    """
    The metaclass for YAMLObject.
    """
    def __new__(cls, *args, **kwargs):
        if cls.yaml_tag:
            cls.yaml_loader.add_constructor(cls.yaml_tag, cls.from_yaml)
            cls.yaml_dumper.add_representer(cls, cls.to_yaml)
        return object.__new__(cls)

class MyYAMLObject(MyYAMLObjectBaseclass):
    """
    An object that can dump itself to a YAML stream
    and load itself from a YAML stream.
    """

    __slots__ = ()  # no direct instantiation, so allow immutable subclasses

    yaml_loader = yaml.Loader
    yaml_dumper = yaml.Dumper

    yaml_tag = None
    yaml_flow_style = None

    @classmethod
    def from_yaml(cls, loader, node):
        """
        Convert a representation node to a Python object.
        """
        return loader.construct_yaml_object(node, cls)

    @classmethod
    def to_yaml(cls, dumper, data):
        """
        Convert a Python object to a representation node.
        """
        return dumper.represent_yaml_object(cls.yaml_tag, data, cls,
                flow_style=cls.yaml_flow_style)


class Monster(MyYAMLObject):
    yaml_tag = '!Monster'

    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks

    def __repr__(self):
        return "{}(name={}, hp={}, ac={}, attacks={}".format(self.__class__.__name__, self.name, self.hp,
                                                             self.ac, self.attacks)


class Dragon(Monster):
    yaml_tag = '!Dragon'

    def __init__(self, name, hp, ac, attacks, energy):
        super(Dragon, self).__init__(name, hp, ac, attacks)
        self.energy = energy

    def __repr__(self):
        return "{}(name={}, hp={}, ac={}, attacks={}, energy={})".format(self.__class__.__name__, self.name, self.hp,
                                                             self.ac, self.attacks, self.energy)

m = Monster(name='Cave spider', hp=[2, 6], ac=16, attacks=['BITE', 'HURT'])
ms = yaml.dump(m)
# print(ms)
d = Dragon(name='Cave spider', hp=[2, 6], ac=17, attacks=['BITE', 'HURT'], energy=5000)
ds = yaml.dump(d)
# print(ds)

print(yaml.load(ms, Loader=yaml.Loader))
print(yaml.load(ds, Loader=yaml.Loader))

Monster(name=Cave spider, hp=[2, 6], ac=16, attacks=['BITE', 'HURT']
Dragon(name=Cave spider, hp=[2, 6], ac=17, attacks=['BITE', 'HURT'], energy=5000)


所以其实我还是没有明白为啥yaml要用metaclass来实现这个功能。

总的来说，metaclass就是一种改变类创建过程的能力，绝大多数情况下都不需要用到它。